In [1]:
gamma=0.25
delta=3

In [2]:
from watermark import Watermark
import torch
from transformers import AutoTokenizer
from watermark import Detector
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch


torch.cuda.set_device(4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer=AutoTokenizer.from_pretrained("/data3/wcr/my_project/models/opt-6.7b")

# if llama
tokenizer.pad_token = tokenizer.eos_token


model= AutoModelForCausalLM.from_pretrained("/data3/wcr/my_project/models/opt-6.7b",torch_dtype=torch.float16).to(device)


# input_text="What is your favorite programming language?"
# input_text="The Linux kernel now includes everything that is needed to use 3D acceleration with all GeForce graphics chips. Drivers have also been added for a Wireless Gigabit chip and a PCIe WLAN chip from Realtek."
input_text="It’s amazing what a difference time can make in the way we perceive certain technologies. A few years ago, the idea of the cloud was terrifying to many people.(give a answer about 200 words) "
wm = Watermark(checkpoint_path="/data3/wcr/LTW/ckpt/tmp/selective_network_epoch0_step2000.pth",device=device,k=6,model=model,tokenizer=tokenizer,min_new_tokens=175, max_new_tokens= 225,embed_unigram_wm=True)

output=wm.generate_watermark(input_text,gamma,delta)
# output=wm.generate_unwatermark(input_text)
print(output)
output=input_text+output[0]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/data3/wcr/LTW/watermark.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.selector_network.load_state_dict(torch.load(checkpoint_path))


['  Now it’s seen by many as inevitable.\nThe fact of the matter is, in my opinion, that in every aspect of technology, but particularly IT, you need to keep an open mind. The fact that over 100 years of my IT experience with my employer I never had a need for an always on, always connected, always connected phone system, I feel like that idea came from just being in the room with the technologists that produced the idea. It was seen as just something that I hadn’t need nor ever seen, like saying a fax machine would not be necessary in twenty years.\nFor sure, but I’m certainly a little hesitant about mobile.  I know I will need a phone, but the idea of keeping it with me at all times, just boggles my mind.\nI feel like I know more of the limitations of mobile technology than I do with desktop technology. I couldn’t, in good faith, argue that mobile technology is worse than the desktop, but I feel like mobile is in a much better place as far as limitations than the']


In [3]:





watermark_detector = Detector(vocab=list(tokenizer.get_vocab().values()),
                                        gamma=gamma,
                                        tokenizer=tokenizer,
                                        z_threshold=4,
                                        model=model,
                                        k=6,
                                        checkpoint_path="/data3/wcr/LTW/ckpt/tmp/selective_network_epoch0_step2000.pth",
                                        embed_unigram_wm=True
                                              
                                        )

# tokenized_input=tokenizer.encode(input_text,  return_tensors='pt').to(device)
# tokenized_output=tokenizer.encode(output[0], return_tensors='pt').to(device)

/data3/wcr/LTW/utils/detect_utils.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.selector_network.load_state_dict(torch.load(checkpoint_path))


In [4]:

tokenized_input=tokenizer.encode(input_text,  return_tensors='pt',add_special_tokens=False).to(device)
tokenized_output=tokenizer.encode(output, return_tensors='pt').to(device)
tokenized_output=tokenized_output[0]
tokenized_input=tokenized_input[0]
detection_result=watermark_detector.detect(tokenized_output,tokenized_input)
detection_result

{'num_tokens_generated': 226,
 'num_tokens_scored': 120,
 'num_green_tokens': 108,
 'watermarking_fraction': 0.5309734513274337,
 'green_fraction': 0.9,
 'z_score': 16.443843832875572,
 'p_value': np.float64(4.6419583101762646e-61),
 'prediction': True,
 'confidence': np.float64(1.0)}

In [5]:
output=wm.generate_unwatermark(input_text)
print(output)
output=input_text+output[0]
tokenized_input=tokenizer.encode(input_text,  return_tensors='pt',add_special_tokens=False).to(device)
tokenized_output=tokenizer.encode(output, return_tensors='pt').to(device)
tokenized_output=tokenized_output[0]
tokenized_input=tokenized_input[0]
detection_result=watermark_detector.detect(tokenized_output,tokenized_input)
detection_result

[" Today cloud computing is the most popular form of computing, yet it is somehow not viewed as a complete abstraction of the underlying hardware.\nI don't see it as a complete abstraction, because then why not just have a big computer and access all the hardware directly? With the cloud, I have to rent a server to store my data on and there are huge, huge costs involved.\nYou make a good point that the cloud isn’t completely ideal. However, there are ways to access that large amount of processing power without having to rent a server, which are getting easier to get into, thanks to cloud computing. As I understand it, there are also ways to minimize the cost of renting a server.  But, since it’s your money, there’s no need for me to explain the process further.  Thanks for your response!</s>"]


{'num_tokens_generated': 177,
 'num_tokens_scored': 83,
 'num_green_tokens': 16,
 'watermarking_fraction': 0.4689265536723164,
 'green_fraction': 0.1927710843373494,
 'z_score': -1.2040760759965081,
 'p_value': np.float64(0.8857199120940613),
 'prediction': False}